In [1]:
import sys
sys.path.append('..')
import numpy as np

In [2]:
### Gamma value generator for benzene
def generate_gamma(norb=6, b=1.4, gamma0=10.84): 
    ang_to_bohr = 1.889726
    har_to_ev = 27.211396
    b_ieV = b*ang_to_bohr/har_to_ev

    gamma = np.zeros((norb,norb))
    for u in range(norb):
        for v in range(norb):
            duv = b*(np.sin(abs(u-v)*np.pi/norb)/np.sin(np.pi/norb))
            gamma[(u,v)] = 1/(1/gamma0 + duv)

    return gamma 

In [3]:
### Building polyene integral arrays
norb = 6

gamma = generate_gamma()

h1 = np.zeros([norb,norb])
for n in range(norb):
    h1[(n,(n+1)%norb)] = -2.5
h1 = h1 + h1.T

h0 = 0
h2 = np.zeros((norb,norb,norb,norb))
H2 = np.zeros([norb,norb])
for n in range(norb):
    h1[(n,n)] -= np.sum(gamma[n,:])
    for m in range(norb):
        h2[(n,n,m,m)] = 0.5*gamma[(n,m)]
        H2[n,m] = h2[(n,n,m,m)]
        h0 += 0.5*gamma[(n,m)]

np.set_printoptions(precision=3)
print('h0=\n',"%0.3f" % h0,'\n')        
print('h1=\n',h1,'\n')
print('h2=\n',H2,'\n')

h0=
 39.962 

h1=
 [[-13.321  -2.5     0.      0.      0.     -2.5  ]
 [ -2.5   -13.321  -2.5     0.      0.      0.   ]
 [  0.     -2.5   -13.321  -2.5     0.      0.   ]
 [  0.      0.     -2.5   -13.321  -2.5     0.   ]
 [  0.      0.      0.     -2.5   -13.321  -2.5  ]
 [ -2.5     0.      0.      0.     -2.5   -13.321]] 

h2=
 [[5.42  0.335 0.199 0.173 0.199 0.335]
 [0.335 5.42  0.335 0.199 0.173 0.199]
 [0.199 0.335 5.42  0.335 0.199 0.173]
 [0.173 0.199 0.335 5.42  0.335 0.199]
 [0.199 0.173 0.199 0.335 5.42  0.335]
 [0.335 0.199 0.173 0.199 0.335 5.42 ]] 



In [6]:
from moha import HamPPP

norb = 6
nelec = 3

beta = -2.5
gamma0 = gamma[0,0]

gamma = generate_gamma(norb)
polyene = HamPPP([(f"C{i}", f"C{i + 1}", 1) for i in range(1, norb)] + [(f"C{norb}", f"C{1}", 1)],
                      alpha=0, beta=beta, gamma=gamma, charges=np.ones(6),
                      u_onsite=np.array([gamma0 for i in range(norb + 1)]))

h1 = polyene.generate_one_body_integral(dense=True, basis='spatial basis')
h2 = polyene.generate_two_body_integral(dense=False, basis='spatial basis')

In [7]:
print('h1=\n',h1,'\n')
print('h2=\n',h2,'\n')

h1=
 [[-13.321  -2.5     0.      0.      0.     -2.5  ]
 [ -2.5   -13.321  -2.5     0.      0.      0.   ]
 [  0.     -2.5   -13.321  -2.5     0.      0.   ]
 [  0.      0.     -2.5   -13.321  -2.5     0.   ]
 [  0.      0.      0.     -2.5   -13.321  -2.5  ]
 [ -2.5     0.      0.      0.     -2.5   -13.321]] 

h2=
   (0, 0)	10.84
  (1, 1)	0.3350642927794263
  (2, 2)	0.1986395532084328
  (3, 3)	0.1728757336055116
  (4, 4)	0.1986395532084328
  (5, 5)	0.3350642927794263
  (7, 7)	10.84
  (8, 8)	0.3350642927794263
  (9, 9)	0.1986395532084328
  (10, 10)	0.1728757336055116
  (11, 11)	0.1986395532084328
  (14, 14)	10.84
  (15, 15)	0.3350642927794263
  (16, 16)	0.1986395532084328
  (17, 17)	0.1728757336055116
  (21, 21)	10.84
  (22, 22)	0.3350642927794263
  (23, 23)	0.1986395532084328
  (28, 28)	10.84
  (29, 29)	0.3350642927794263
  (35, 35)	10.84 

